In [ ]:
import pandas as pd
import numpy as np
import nltk
import re

##Importação dos dados
Inicialmente, será importado dados para treino e teste

In [ ]:
#Importando dados de treino
train = pd.read_excel('/content/drive/My Drive/Colab Notebooks/handtalk/Português/emotions-pt2.xlsx')
##train = pd.read_excel('/content/drive/My Drive/Colab Notebooks/handtalk/Português/emotions-sem-love-pt.xlsx')
#train = pd.read_excel('datasets/emotions-pt2.xlsx')

#Misturando os dados
train = train.sample(frac=1).reset_index(drop=1)

#Substituindo 'alegria' por 'felicidade'
train['sentimento'] = train['sentimento'].replace('alegria', 'felicidade')

train.head()

,conteúdo,sentimento
0,eu acho que o homem sabe como fazer cada um de...,felicidade
1,me sinto espancado e machucado por sua asperez...,tristeza
2,Eu me sinto honrado em ser testemunha do proce...,felicidade
3,Já estou me sentindo sentimental sobre seu tem...,tristeza
4,eu abro meus olhos de manhã meu coração parece...,tristeza


In [ ]:
#Importando dados para teste
test = pd.read_excel('/content/drive/My Drive/Colab Notebooks/handtalk/Português/emotions-test-pt.xlsx')
#test = pd.read_excel('datasets/emotions-test-pt.xlsx')

#Substituindo 'alegria' por 'felicidade'
test['sentimento'] = test['sentimento'].replace('alegria', 'felicidade')

test.head()

,conteúdo,sentimento
0,"estou me sentindo um tanto podre, então não so...",tristeza
1,estou atualizando meu blog porque me sinto uma...,tristeza
2,eu nunca a separo de mim porque eu nunca quero...,tristeza
3,saí com meu buquê de tulipas vermelhas e amare...,felicidade
4,Eu estava me sentindo um pouco vaidoso quando ...,tristeza


In [ ]:
train['sentimento'].value_counts()

felicidade    5362
tristeza      4666
raiva         2308
Name: sentimento, dtype: int64

In [ ]:
#Retirando dados duplicados do dataset de treino
print(f"Antes: {train['conteúdo'].count()}")

train.drop_duplicates(inplace=True)
print(f"Depois: {train['conteúdo'].count()}")

Antes: 12336
Depois: 12321


In [ ]:
#Separando as sentenças dos sentimentos para o treino
X_train = train['conteúdo']
Y_train = train['sentimento']

In [ ]:
#Separando as sentenças dos sentimentos para o teste
X_test = test['conteúdo']
Y_test = test['sentimento']

## Preprocessamento
Nesta etapa será definida funções para limpar os dados, além de remover as stopwords

In [ ]:
#Criação da função de Stemming, para reduzir as palavras em radicais
def criar_radicais(conteudo):
  stemmer = nltk.stem.RSLPStemmer()
  palavras = [stemmer.stem(p) for p in conteudo.split()]
  return " ".join(palavras)

In [ ]:
#Efetiva-se um transformador customizado para o pré-processamento
from sklearn.base import BaseEstimator, TransformerMixin

class CustomTransformer(BaseEstimator, TransformerMixin):
  def __init__(self):
    #Baixa as stopwords
    nltk.download('stopwords')

  def fit(self, X, y=None):
    return self

  def transform(self, X, y=None):
    if isinstance(X, pd.core.series.Series) or isinstance(X, pd.core.frame.DataFrame):
      return X.apply(lambda x: self.preprocessador(x))
    if isinstance(X, list):
      return map(lambda x: self.preprocessador(x), X)

  def preprocessador(self, instancia):
    #Removendo stopwords
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    stopwords.remove('não')

    #Removendo caracteres indesejados
    instancia = [p.lower() for p in instancia.split() if p.lower() not in stopwords]
    instancia = " ".join(instancia)

    instancia = re.sub("[,.;/\[\]?!\'\"]", '', instancia)
    return instancia

  pre = CustomTransformer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Classificação
Será utilizado SVC e Naivy Bayes (MultinomialNB) para realizar o fit dos dados, além da tokenização dos dados. Por fim, será validado os modelos com o Cross Validation.

In [ ]:
#Criando um pipeline para Tokenização e Classificação
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

#Pipeline para Naivy Bayes
pipe_nb = Pipeline([
    ('vetor', CountVectorizer(analyzer='word')),
    ('nb', MultinomialNB())
])

#Pipeline para SVC
pipe_svc = Pipeline([
    ('vetor', CountVectorizer(analyzer='word')),
    ('svc', SVC(kernel='linear', probability=True))
])

In [ ]:
#Pré-processando os dados
X_train = pre.transform(X_train)
X_test = pre.transform(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score

score_nb = cross_val_score(pipe_nb, X_train, Y_train, cv=6)
score_svc = cross_val_score(pipe_svc, X_train, Y_train, cv=6)

In [ ]:
print(f'MultinomialNB: {round(score_nb.mean(), 3)}\nSVC: {round(score_svc.mean(), 3)}')
#Bons resultados, por agora

MultinomialNB: 0.828
SVC: 0.856


## Predições
Será utilizado os modelos que foram treinados, e analisar qual deles terá um desempenho melhor.

In [ ]:
#Fitando os dados com suas classificações
pipe_nb.fit(X_train, Y_train)
pipe_svc.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vetor',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1),
                                 preprocessor=<function preprocessador at 0x7f6f3aa14e18>,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='linear', max_iter=-1,
                  

In [ ]:
#Realizando as predições para ambos modelos
pred_nb = pipe_nb.predict(X_test)
pred_svc = pipe_svc.predict(X_test)

In [ ]:
#Medindo o desempenho do modelo para os dados de teste
from sklearn.metrics import accuracy_score, classification_report

def metricas(clf, real, predito, proba=False):
  classes = Y_train.unique().tolist()
  
  print(f"""{clf}
Acurácia: {round(accuracy_score(real, predito), 3)}
Relatório: 
{classification_report(real, predito, classes)}""")

metricas('Naivy Bayes', pred_nb, Y_test)
metricas('SVC', pred_svc, Y_test)

Naivy Bayes
Acurácia: 0.836
Relatório: 
              precision    recall  f1-score   support

  felicidade       0.92      0.86      0.89      1501
       raiva       0.56      0.88      0.69       350
    tristeza       0.87      0.80      0.83      1229

    accuracy                           0.84      3080
   macro avg       0.78      0.85      0.80      3080
weighted avg       0.86      0.84      0.84      3080

SVC
Acurácia: 0.866
Relatório: 
              precision    recall  f1-score   support

  felicidade       0.93      0.88      0.90      1466
       raiva       0.78      0.82      0.80       522
    tristeza       0.83      0.86      0.85      1092

    accuracy                           0.87      3080
   macro avg       0.85      0.86      0.85      3080
weighted avg       0.87      0.87      0.87      3080



In [ ]:
#Agora um teste com frases próprias
frases = ['Passei na prova da faculdade', #felicidade
         'Estou cansado, passei o dia todo trabalhando', #tristeza
         'Vou bater no próximo que aparecer na minha frente', #raiva
         'Tenho as melhores pessoas ao meu lado', #felicidade
         'Confesso que tenho vontade de espancar certa pessoa', #raiva
         'Estou muito feliz'] #felicidade

#Pré-processamento
frases = pre.transform(frases)

#Probabilidade das classes
proba_nb = pipe_nb.predict_proba(frases)
proba_svc = pipe_svc.predict_proba(frases)

classes = ['felicidade', 'raiva', 'tristeza']
print(f'NB:\nClassificações: {[classes[i] for i in np.argmax(proba_nb, axis=1)]}\nProbabilidades: {proba_nb}')
print(f'\n\nSVC:\nClassificações: {[classes[i] for i in np.argmax(proba_svc, axis=1)]}\nProbabilidades: {proba_svc}')

NB:
Classificações: ['felicidade', 'tristeza', 'felicidade', 'felicidade', 'raiva', 'felicidade']
Probabilidades: [[0.58955438 0.07701973 0.3334259 ]
 [0.13648878 0.02379798 0.83971324]
 [0.58431933 0.17006785 0.24561282]
 [0.47792894 0.18461559 0.33745547]
 [0.44798615 0.5141294  0.03788446]
 [0.78156837 0.05636907 0.16206256]]


SVC:
Classificações: ['felicidade', 'tristeza', 'raiva', 'felicidade', 'raiva', 'felicidade']
Probabilidades: [[0.45948061 0.19182376 0.34869562]
 [0.04152714 0.0545191  0.90395376]
 [0.16166227 0.68532826 0.15300946]
 [0.37131267 0.31647414 0.31221319]
 [0.2289892  0.71077128 0.06023952]
 [0.88129333 0.05622704 0.06247963]]


## Fine Tuning
Apesar dos resultados serem satisfatórios, é possível melhora. Para isto, seraárealizado ajuste de hiperparâmetros, com a procura da melhor combinação destes, utilizando o GridSearchCV.

In [ ]:
#Introdução do TfidfTransformer, para aumentar os pesos das palavras mais importantes
from sklearn.feature_extraction.text import TfidfTransformer

#Pipeline para Naivy Bayes
pipe_nb2 = Pipeline([
    ('vetor', CountVectorizer(analyzer='word', preprocessor=preprocessador)),
    ('tfidf', TfidfTransformer()),
    ('nb', MultinomialNB())
])

#Pipeline para SVC
pipe_svc2 = Pipeline([
    ('vetor', CountVectorizer(analyzer='word', preprocessor=preprocessador)),
    ('tfidf', TfidfTransformer()),
    ('svc', SVC(kernel='linear', probability=True))
])

In [ ]:
#Função para automatizar etapas finais
def resultados_fine_tuning(X_train=X_train, Y_train=Y_train, X_test=X_test, Y_test=Y_test, frases=frases):
  pipe_nb2.fit(X_train, Y_train)
  pipe_svc2.fit(X_train, Y_train)

  pred_nb = pipe_nb2.predict(X_test)
  pred_svc = pipe_svc2.predict(X_test)

  metricas('Naivy Bayes', pred_nb, Y_test)
  metricas('SVC', pred_svc, Y_test)

  frases_nb = pipe_nb.predict(frases)
  frases_svc = pipe_svc.predict(frases)

  print(f'\n\nPredições Customizadas:\nNB: {frases_nb}\nSVC: {frases_svc}')

resultados_fine_tuning()

Naivy Bayes
Acurácia: 0.783
Relatório: 
              precision    recall  f1-score   support

    tristeza       0.84      0.78      0.81      1223
       raiva       0.23      0.98      0.38       130
     alegria       0.95      0.77      0.85      1727

    accuracy                           0.78      3080
   macro avg       0.68      0.84      0.68      3080
weighted avg       0.88      0.78      0.81      3080

SVC
Acurácia: 0.873
Relatório: 
              precision    recall  f1-score   support

    tristeza       0.85      0.87      0.86      1110
       raiva       0.72      0.89      0.80       446
     alegria       0.95      0.87      0.91      1524

    accuracy                           0.87      3080
   macro avg       0.84      0.88      0.86      3080
weighted avg       0.88      0.87      0.88      3080



Predições Customizadas:
NB: [[0.5272885  0.10762202 0.36508948]
 [0.31179607 0.06068371 0.62752022]
 [0.51636067 0.14674049 0.33689884]
 [0.50212991 0.13716251 0.36

Resultado parecido com o original, com queda de rendimento para classificar 'raiva', além de aumentar o tempo do fit.

In [ ]:
#Utilizando o GridSearchCV para realizar o ajuste dos parâmetros
from sklearn.model_selection import GridSearchCV

#Parâmetros para serem analisados
params = {
    'vetor__ngram_range': [(1, 1), (1, 2), (2,2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': (None, 'l1', 'l2')
}

#Parâmetros exclusivo do Naivy Bayes
params_nb = {'nb__alpha': (1e-2, 1e-3), 'nb__fit_prior': [True, False]}
params_nb.update(params)

#Parâmetros exclusivo do SVC
params_svc = {
    'svc__kernel': ['linear', 'rbf'],
    'svc__gamma': [0.1, 1, 10],
    'svc__C': [0.1, 1, 10]
}
params_svc.update(params)

#Pipeline para Naivy Bayes
pipe_nb2 = Pipeline([
    ('vetor', CountVectorizer(analyzer='word', preprocessor=preprocessador)),
    ('tfidf', TfidfTransformer()),
    ('nb', MultinomialNB())
])

In [ ]:
#GridSearch para Naivy Bayes
nb_gs = GridSearchCV(pipe_nb2, params_nb, cv=3, verbose=2)
nb_gs = nb_gs.fit(X_train, Y_train)

In [ ]:
print(f'Score: {nb_gs.best_score_}')
print(f'Parâmetros: {nb_gs.best_params_}')

Score: 0.833860887914942
Parâmetros: {'nb__alpha': 0.01, 'nb__fit_prior': False, 'tfidf__norm': 'l1', 'tfidf__use_idf': False, 'vetor__ngram_range': (1, 2)}


Para uma SVC, o treinamento estava demorando bastante. Para tentar reduzir o tempo de treinamento, será utilizado o BaggingClassifier, em que terá 10 SVCs.

In [ ]:
#Utilização do Bagging Classifier
from sklearn.ensemble import BaggingClassifier

#Quantidade de estimadores
n_est = 10

#Parâmetros exclusivo do SVC
params_svc = {
    'clf__base_estimator__kernel': ['linear', 'rbf'],
    'clf__base_estimator__gamma': [0.1, 1, 10],
    'clf__base_estimator__C': [0.1, 1, 10]
}
params_svc.update(params)

clf = BaggingClassifier(SVC(probability=True), max_samples=1.0/n_est, n_estimators=n_est)

#Pipeline para SVC
pipe_svc2 = Pipeline([
    ('vetor', CountVectorizer(analyzer='word', preprocessor=preprocessador)),
    ('tfidf', TfidfTransformer()),
    ('clf', clf)
])

In [ ]:
#GridSearch para SVC
svc_gs = GridSearchCV(pipe_svc2, params_svc, cv=3, verbose=2, n_jobs=-1)
svc_gs = svc_gs.fit(X_train, Y_train)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 22.5min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 52.2min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 94.2min
[Parallel(n_jobs=-1)]: Done 972 out of 972 | elapsed: 143.2min finished


In [ ]:
print(f'Score: {svc_gs.best_score_}')
print(f'Parâmetros: {svc_gs.best_params_}')

Score: 0.7940102264426588
Parâmetros: {'clf__base_estimator__C': 0.1, 'clf__base_estimator__gamma': 1, 'clf__base_estimator__kernel': 'linear', 'tfidf__norm': None, 'tfidf__use_idf': True, 'vetor__ngram_range': (1, 1)}


Surpreendentemente, o classificador de SVC apresentou resultados piores após o ajuste de parâmetros

## Salvando o modelo
Para que a utilização do modelo escolhido seja mais fácil e prática, o modelo será salvo, utilizando o joblib.

In [ ]:
import joblib

#Salvando o modelo com melhor performance
joblib.dump(pipe_svc, '/content/drive/My Drive/Colab Notebooks/handtalk/model.sav')
#joblib.dump(pipe_svc, 'model/model.sav')